## Next Steps

## History
- 20.11.19: replace missing 3YPerf-values by 3YPerf-mean
- 20.11.19: add parameter for 3MPerf calculation by quantiles or absolute values
- 19.11.19: removed missing values addition
- 17.11.19: added Depot as bubble
- 17.11.19: 3M Performance now in 20% quantiles
- 16.11.19: initial version

In [1]:
import datetime
import pandas as pd
import sys
from numpy import nan as NA

In [2]:
useQuantiles = True
if len(sys.argv) > 1:
    arg = sys.argv[1] 
else:
    arg=''
if 'absolut' in arg:
        useQuantiles = False

In [3]:
def floatconv(val):
    try:
        if val.strip():
            return float(val.replace('.','').replace(',','.'))
        else:
            return 0
    except ValueError as ve:
        print("VALUE NOT USABLE for floatconv: #{}#".format(val))

In [4]:
def percentconv(val):
    try:
        if '%' in val:
            return floatconv(val.replace('%', ''))
        else:
            return NA
    except ValueError as ve:
        print("VALUE NOT USABLE for percentconv: #{}#".format(val))

In [5]:
intconv = lambda val: 0 if len(str(val)) < 2 else float(str(val).replace('.',''))
converter = {'Aktuell':floatconv, 'Wert in EUR':floatconv, 'Perf. 3 Monate':percentconv, \
             'Perf. 1 Jahr':percentconv, 'Perf. 3 Jahre':percentconv}

In [6]:
wkn2short = pd.read_csv('wkn2names.csv', header=None, sep=':', index_col=0, squeeze=True).to_dict()

def shortname(longname):
    return wkn2short[longname]

In [13]:
#filename = "musterdepot_Komplett_meineuebersicht_20191112_1043.csv"
filename = "musterdepot_Komplett_meineuebersicht.csv"
#data = pd.read_csv(filename, sep=";", header=2, encoding="iso-8859-1", converters = converter, usecols=[0,2,4,19,20,21])
data = pd.read_csv(filename, sep=";", header=2, encoding="iso-8859-1", converters = converter, usecols=["Stück","WKN","Aktuell","Perf. 3 Monate","Perf. 1 Jahr"])
data['Wert'] = data['Stück']*data['Aktuell']
data['Name'] = data['WKN'].apply(lambda x: shortname(x[0:23]))

In [14]:
a='Amundi Index Solutions'
b=a.replace('Amundi Index Solutions','AIS')
b
data

,Stück,WKN,Aktuell,Perf. 3 Monate,Perf. 1 Jahr,Wert,Name
0,140,A0KRKM,10.600,-59.97,-62.39,1484.000,Brent
1,150,978041,29.412,-36.61,-35.24,4411.800,LBBW Div Euro
2,40,A2H577,88.960,-3.82,15.88,3558.400,AIS NASDAQ
3,250,702979,103.960,-23.00,-15.27,25990.000,Coba DAX
4,540,973277,91.955,-30.24,-27.84,49655.700,FF Nordi
5,60,716460,107.420,-14.08,4.95,6445.200,SAP
6,130,ETF908,22.910,-6.79,3.59,2978.300,Lyxor TecDAX
7,25,976986,337.669,-24.85,-16.94,8441.725,DWS AS DL
8,50,A0MX8G,81.130,-5.56,7.64,4056.500,OEkwrld Klima
9,2200,593397,26.535,-8.86,1.42,58377.000,iShares TecDAX


In [9]:
# add missing values: (better than adding missing values is selecting a better stock exchange)
#Nasdaq Performance
#data.loc[0,'Perf. 3 Jahre'] = 61
#TecDax Performance
#data.loc[2,'Perf. 3 Jahre'] = 75
#A2ANH0 Performance
#data.loc[9,'Perf. 3 Jahre'] = 55

# or simply replace by mean value:
data['Perf. 3 Jahre'].fillna(data['Perf. 3 Jahre'].mean(),inplace=True)

In [10]:
data

,Stück,WKN,Aktuell,Perf. 3 Monate,Perf. 1 Jahr,Perf. 3 Jahre,Wert,Name
0,100,A2JSDA,52.622,-12.95,0.04,5.881429,5262.200,AIS MSCI SRI
1,40,A2H577,83.310,-7.06,12.64,49.840000,3332.400,AIS NASDAQ
2,699,555750,12.470,-15.97,-16.50,-22.730000,8716.530,DTE
3,25,976986,332.165,-24.87,-14.64,-12.170000,8304.125,DWS AS DL
4,1800,973270,13.127,-22.65,-13.47,-13.180000,23628.600,FF Euro Growth
5,425,A1W4TN,32.840,-10.57,9.89,52.040000,13957.000,FF - Gl Tech
6,200,973266,29.096,-18.69,-19.54,-18.290000,5819.200,FF Malaysia
7,540,973277,91.565,-31.31,-25.71,-30.120000,49445.100,FF Nordi
8,470,A2ANH0,6.324,-19.06,-8.24,17.110000,2972.280,iShares Automation
9,2200,593397,25.460,-9.83,1.96,34.740000,56012.000,iShares TecDAX


In [11]:
def perf2String(val,quantiles):
    if val < quantiles[0.2]:
        return 'lowest'
    if val < quantiles[0.4]:
        return 'low'
    if val < quantiles[0.6]:
        return 'mid'
    if val < quantiles[0.8]:
        return 'high'
    return 'highest'

def perf2Stringabsolut(val):
    if val < -4:
        return 'lowest'
    if val < 0:
        return 'low'
    if val < 2:
        return 'mid'
    if val < 4:
        return 'high'
    return 'highest'


In [12]:
if useQuantiles == True:
    quantiles = data['Perf. 3 Monate'].quantile([0.2,0.4,0.6,0.8])
    data['Perf3MString'] = data['Perf. 3 Monate'].apply(lambda x: perf2String(x,quantiles))
else:
    data['Perf3MString'] = data['Perf. 3 Monate'].apply(lambda x: perf2Stringabsolut(x))

In [13]:
data['Value3MAgo'] = data['Wert']/(1+data['Perf. 3 Monate']/100)
data['Value1YAgo'] = data['Wert']/(1+data['Perf. 1 Jahr']/100)
data['Value3YAgo'] = data['Wert']/(1+data['Perf. 3 Jahre']/100)

valuetoday = data['Wert'].sum()
v3mago = data['Value3MAgo'].sum()
v1yago = data['Value1YAgo'].sum()
v3yago = data['Value3YAgo'].sum()
p3m = (valuetoday/v3mago-1)*100
p1y = (valuetoday/v1yago-1)*100
p3y = (valuetoday/v3yago-1)*100
print("Portfolio total value: {0:7.2f}, 3-month-performance: {1:3.2f}% , 1Y-perf: {2:3.2f}%, 3Y-perf: {3:3.2f}%"
      .format(valuetoday,p3m,p1y,p3y))

Portfolio total value: 263712.91, 3-month-performance: -20.19% , 1Y-perf: -10.27%, 3Y-perf: -4.43%


In [14]:
def rd(val):
    return int(round(val))

In [15]:
#['Name','3JPerf','1J Perf','3M Perf','EUR'],
#['Nasdaq',32,22,'high',3214],

#lines with values:
values=""
for i, (index, row) in enumerate(data.iterrows()):
    values += "['"+row['Name']+"'," \
    +str(rd(row['Perf. 3 Jahre']))+"," \
    +str(rd(row['Perf. 1 Jahr']))+",'" \
    +row['Perf3MString']+"'," \
    +str(rd(row['Wert']))+"]," \
    +'\n'

#Depot line with 1y and 3y performance and 3m as part of the name (special color)    
values += "['Depot 3M:"+str(round(p3m,2))+"%'," \
+str(rd(p3y))+"," \
+str(rd(p1y))+"," \
+"'Depot','" \
+str(50000) \
+"']\n"

In [16]:
# read template, replace placeholder and write output file:
with open('portfolioPerformance_in.html','rt') as fin, open('portfolioPerformance.html','wt') as fout:
    for line in fin:
        if '#$0' in line:
            line = line.replace('#$0',values) 
        if '#$1' in line:
            today = datetime.date.today()
            line = line.replace('#$1',today.strftime('%d.%m.%Y'))
        if '#$2' in line:
            if useQuantiles == True:
                line = line.replace('#$2',str(quantiles.values))
            else:
                line = line.replace('#$2','[-4;0;2;4[')
        fout.write(line)